# Bias and variance and extending JIDT code

This activity illustrates bias and variance of MI estimates, and gets you started extending the code generated by the JIDT AutoAnalyser.

1. Start by generating the code again for the demonstration of the Discrete MI on slides 18/19 of the "Introduction to JIDT" lecture slides. Make sure that you have:
    * Set the `base` back to 2 for binary data, and
    * Set the `time difference` property back to 0 (so we look at the zero lag MI)
2. From the generated Python tab from the AutoAnalyer panel (or from the file `demos/AutoAnalyser/GeneratedCalculator.py`) copy and paste the import lines and the lines to start the JVM into the first code cell below.

In [1]:
# Paste the import lines and the lines to start the JVM in this code cell:

from jpype import *
import numpy
import sys
# Our python data file readers are a bit of a hack, python users will do better on this:
sys.path.append("/home/joseph/JIDT/infodynamics-dist-1.6/demos/python")
import readIntsFile

if (not isJVMStarted()):
    # Add JIDT jar library to the path
    jarLocation = "/home/joseph/JIDT/infodynamics-dist-1.6/infodynamics.jar"
    # Start the JVM (add the "-Xmx" option with say 1024M if you get crashes due to not enough memory space)
    startJVM(getDefaultJVMPath(), "-ea", "-Djava.class.path=" + jarLocation)

Paste the remaining code making calculations etc (from the step 0 comments onwards) into the next code cell below.<br/>
Splitting the two parts of the code means that we can re-run the code making calculations without having to re-run the code starting the JVM. You can re-run the above code (since it detects and skips starting the JVM if its already running), but it's not necessary. You might want to take this approach whenever you use JIDT in notebooks.<br/>
(Alternatively you can just work in a new `.py` file instead of this notebook, which can be placed anywhere).

3. Run the two cells to make sure the code still works ok.

In [2]:
# Paste the remaining code making calculations etc in this code cell:

numSamples = 10;
numCalcs = 50;
results = numpy.zeros((numCalcs));

# 1. Construct the calculator:
calcClass = JPackage("infodynamics.measures.discrete").MutualInformationCalculatorDiscrete
calc = calcClass(2, 2, 0)

for i in range(numCalcs):
    source = numpy.random.randint(0, 2, 10);
    destination = source;
    
    # 2. No other properties to set for discrete calculators.
    # 3. Initialise the calculator for (re-)use:
    calc.initialise()
    # 4. Supply the sample data:
    calc.addObservations(source, destination)
    # 5. Compute the estimate:
    result = calc.computeAverageLocalOfObservations()
    results[i] = result
    
    print("MI_Discrete(col_0 -> col_1) = %.4f bits" %
        (result))

print("Results have mean %.3f bits (bias of %.3f bits) and variance %.3f bits" % (numpy.mean(results), numpy.mean(results) - 1, numpy.var(results)))

MI_Discrete(col_0 -> col_1) = 0.9710 bits
MI_Discrete(col_0 -> col_1) = 0.9710 bits
MI_Discrete(col_0 -> col_1) = 0.9710 bits
MI_Discrete(col_0 -> col_1) = 0.9710 bits
MI_Discrete(col_0 -> col_1) = 0.9710 bits
MI_Discrete(col_0 -> col_1) = 1.0000 bits
MI_Discrete(col_0 -> col_1) = 0.7219 bits
MI_Discrete(col_0 -> col_1) = 1.0000 bits
MI_Discrete(col_0 -> col_1) = 0.9710 bits
MI_Discrete(col_0 -> col_1) = 0.8813 bits
MI_Discrete(col_0 -> col_1) = 1.0000 bits
MI_Discrete(col_0 -> col_1) = 0.4690 bits
MI_Discrete(col_0 -> col_1) = 0.9710 bits
MI_Discrete(col_0 -> col_1) = 0.9710 bits
MI_Discrete(col_0 -> col_1) = 0.7219 bits
MI_Discrete(col_0 -> col_1) = 0.8813 bits
MI_Discrete(col_0 -> col_1) = 0.8813 bits
MI_Discrete(col_0 -> col_1) = 0.7219 bits
MI_Discrete(col_0 -> col_1) = 0.9710 bits
MI_Discrete(col_0 -> col_1) = 0.9710 bits
MI_Discrete(col_0 -> col_1) = 0.9710 bits
MI_Discrete(col_0 -> col_1) = 0.9710 bits
MI_Discrete(col_0 -> col_1) = 0.8813 bits
MI_Discrete(col_0 -> col_1) = 0.97

4. Edit step 0 of the code which loads the data in to the `source` and `destination` variables:
    1. First remove the lines where the file is loaded.
    2. Next, change the assignment of the source variable to be an array of 10 random bits: `source = numpy.random.randint(0, 2, 10);` Note that this returns an array of 0's and 1's as required by JIDT.
    3. Finally, change the assignment of the destination variable to be a copy of the source: `destination = source;`
5. Congratulations, you have made your first extension of the automatically generated JIDT code! Now run the code with these changes.
6. Note the result. Was it the full 1 bit of shared information the we would expect for copied random bits?
7. Run the code several more times and note the results. Are they always the same or do they vary? Why is this?
8. Capture the results of running the code several times (say 10 times) into an array and measure the mean and variance of the results. (_Hint_: in Python you can create an empty array as `results = numpy.zeros(10);`, and then assign into this as say `results[0] = result1;`. You would be best to use a `for` loop to run the code 10 times).
    1. Compute the bias as the difference between the mean empirical result and the expected result. It is quite large here because we have computed the empirical results from so few samples (10). In the lecture we noted that MI is typically biased upwards, which referred to situations where variables don't actually share any information; where variables do indeed share information, the MI can be biased downwards as is the case here.
    2. Also try to increase the number of samples (e.g. upwards from 10 random bits to 100) and see how the bias and variance change.